In [37]:
# Load libraries
import csv
import pandas as pd
import numpy as np
import pdb
import copy
import scipy.spatial.distance
import datetime
import gc
from scipy.stats import ttest_ind

In [56]:
# PARAMETERS

# Posible distances: mean_absolute_difference; mean_squared_differences; absolute_difference; squared_differences; cosine; euclidean
distance_metric = 'mean_absolute_difference'

# Assignments to keep (we can use all assignments)
selectedItems = ['Quiz_1_2', 'Quiz_3', 'Quiz_4_5', 'Quiz_6', 'Quiz_7', 'Quiz_8', 'Quiz_9_10', 'Quiz_11', 'Quiz_12', 'Quiz_13', 'Quiz_14', 'Midterm_randxyzHP3X5J4M','Posttest_randxyzX9LZKV0I']

# Mean percentage of items in common
minPercentageCommonItems = 20

In [59]:
submission_matrix = pd.read_csv("example_submission_matrix.csv", index_col=0)

In [40]:
# These lines could be used to use only a selection of assignments
colsToKeep = []
"""
for col in submission_matrix.columns:
    colsToKeep.append(col in selectedInputFields)

submission_matrix = submission_matrix.iloc[:, colsToKeep]
"""

'\nfor col in submission_matrix.columns:\n    colsToKeep.append(col in selectedInputFields)\n\nsubmission_matrix = submission_matrix.iloc[:, colsToKeep]\n'

In [61]:
percentageItems = round(100*(pd.notna(submission_matrix).sum(axis = 1)/len(submission_matrix.columns)))
# print((percentageItems > minPercentageItems).sum())
# students to keep with a minItems submitted
minItems_submission_matrix = submission_matrix[percentageItems > minPercentageCommonItems]

In [62]:
# Calculates the distance between two arrays of submissions timestamps:
# arg1: Array of submissions 1
# arg2: Array of submissions 2
# distance_metric: The distance metric we want to compute choice in {absolute_difference, squared_differences, cosine, euclidean}
# return: The distance beteween the two arrays of submissions for the chosen metric
def compute_distance(submissions1, submissions2, distance_metric='absolute_difference', onlyCommonItems=True):
    
    # Here I adapt the two arrays of submissions to take into account only the common items
    if(onlyCommonItems):
        commonMask = (pd.notna(submissions1) & pd.notna(submissions2))      
        percentageCommonItems = 100*(sum(commonMask)/len(submissions1))
        submissions1 = pd.to_datetime(submissions1[commonMask])
        submissions2 = pd.to_datetime(submissions2[commonMask])
        
    if(percentageCommonItems < 20):
        return np.nan    
    
    if(distance_metric == 'absolute_difference'):
        distance = np.absolute(submissions1 - submissions2).sum()
    elif(distance_metric == 'mean_absolute_difference'):
        distance = np.absolute(submissions1 - submissions2).sum()/len(submissions1)
    elif(distance_metric == 'squared_differences'):
        distance = pow((submissions1 - submissions2),2).sum()
    elif(distance_metric == 'mean_squared_differences'):
        distance = pow((submissions1 - submissions2),2).sum()/len(submissions1)
    elif(distance_metric == 'euclidean'):
        distance = scipy.spatial.distance.euclidean(submissions1, submissions2)
    elif(distance_metric == 'cosine'):
        distance = scipy.spatial.distance.cosine(submissions1, submissions2)
    
    return round(distance.total_seconds()/60)

In [46]:
#(submission_matrix_noNA.iloc[1] - submission_matrix_noNA.iloc[2])/1000
#scipy.spatial.distance.euclidean(submission_matrix_noNA.iloc[1], submission_matrix_noNA.iloc[2])
# get lower diagonal http://stackoverflow.com/questions/26477901/python-pand3s-get-elements-index-col-below-diagonal-in-dataframe
print(compute_distance(minItems_submission_matrix.iloc[1], minItems_submission_matrix.iloc[4], 'mean_absolute_difference'))
#np.absolute(minItems_submission_matrix.iloc[1] - minItems_submission_matrix.iloc[2]).sum()/3600)/len(submission_matrix_noNA.iloc[2])
#print compute_distance(submission_matrix_noNA.iloc[0]/3600, submission_matrix_noNA.iloc[5]/3600)

#print (datetime.datetime.fromtimestamp(int(submission_matrix_noNA.loc['0023d437fc00a981e344ec69103b8f06955b675b'][0])) - datetime.datetime.fromtimestamp(int(submission_matrix_noNA.loc['0027319f7b8f28b9812ab075c6ca9678c6188b8d'][0]))).total_seconds() 

1784


In [63]:
# Creating distances matrix in minutes
distance_matrix = pd.DataFrame()
print("Starting...")
for index, studentCol in enumerate(minItems_submission_matrix.index):
    if (index % 50) == 0:
        print(index)
    distancesStudentCol = []
    for studentRow in minItems_submission_matrix.index:
        distancesStudentCol.append(compute_distance(minItems_submission_matrix.loc[studentCol], 
                                                    minItems_submission_matrix.loc[studentRow], 
                                                    distance_metric))
    distance_matrix[studentCol] = distancesStudentCol

print("...Finished!")
distance_matrix.index = minItems_submission_matrix.index

Starting...
0
50
100
...Finished!


In [18]:
# Save distance matrix
distance_matrix.to_csv("distance_matrix_" + distance_metric + "_" + selectedItems +".csv", sep=";", header=True, na_rep=np.nan, index=False)

In [ ]:
# Load distance matrix
distance_matrix = pd.read_csv("distance_matrix_" + distance_metric + "_" + selectedItems +".csv", sep=";")

In [66]:
# Create mask for unique triplets {student1, student2, distance} --
mask = np.ones(distance_matrix.shape, dtype=bool)
mask[np.triu_indices(len(distance_matrix))] = False
#distance_matrix = distance_matrix[mask] --> # THIS WAS THROWING A MEMORY LEAK, IM DOING A SIMPLE LOOP INSTEAD

In [67]:
counter = 0
studentIDs = list(distance_matrix.columns.values)

students1 = []
students2 = []
distances = []

tripletsDataframe = pd.DataFrame()
print("Starting...")

for index, celBool in np.ndenumerate(mask):
    counter += 1
    if (counter % 200000) == 0:
        print(str((counter/float(distance_matrix.size))*100) + "%")
    if(celBool):
        students1.append(studentIDs[index[0]])
        students2.append(studentIDs[index[1]])
        distances.append(distance_matrix.iloc[index[0],index[1]])

tripletsDataframe['students1'] = students1
tripletsDataframe['students2'] = students2
tripletsDataframe['distances'] = distances

print("...Finished!")

Starting...
...Finished!


In [70]:
tripletsDataframe[tripletsDataframe['distances'] < 30]

,students1,students2,distances
116,27,20,23.0
896,85,67,4.0
914,86,20,13.0
970,88,45,16.0
1046,91,20,9.0
1078,91,86,4.0
1286,104,20,8.0
1318,104,86,5.0
1321,104,91,1.0
1718,119,12,4.0


In [22]:
tripletsDataframe.to_csv("triplets_matrix_" + distance_metric + "_" + selectedItems +".csv", sep=";", header=True, index=False)

In [26]:
tripletsDataframe[tripletsDataframe['distances'] < 90].to_csv("selected_triplets_" + distance_metric + "_" + selectedItems +".csv", sep=";", header=True, index=False)